# 第7回課題
2段階の train, valid, test にわける交差検証の自前の実装
（パッケージにしたり関数にしたりするのは自由です．解答はしていません）

https://scikit-learn.org/stable/modules/cross_validation.html　で様々なパターンについて述べられています

# 注意事項
train, valid, test で取ってくるサンプル（id）が一致しないということを必ず確認してください

In [69]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [70]:
data = load_wine()

In [71]:
wine_df = pd.DataFrame(min_max_scaler.fit_transform(data["data"]), columns=data["feature_names"])
wine_df = pd.concat([wine_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
print(wine_df.shape)
wine_df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0


# モデル（今回は決定木を利用）

In [72]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0)

# seed の設定

In [73]:
seed = 0
#def setseed():
#    random.seed(seed)
#    np.random.seed(seed)

# k-fold の k は変数で扱うこと

In [74]:
k = 5

# train (valid 含む), test の分割

In [75]:
wine_df_s = wine_df.sample(frac=1, random_state=seed)
wine_df_s.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
54,0.713158,0.183794,0.475936,0.298969,0.521739,0.558621,0.540084,0.150943,0.381703,0.389932,0.357724,0.706960,0.557775,0
151,0.463158,0.381423,0.598930,0.587629,0.456522,0.172414,0.215190,0.207547,0.268139,0.812287,0.000000,0.073260,0.144080,2
63,0.352632,0.077075,0.427807,0.432990,0.184783,0.868966,0.582278,0.113208,0.460568,0.270478,0.601626,0.586081,0.101284,1
55,0.665789,0.195652,0.588235,0.510309,0.500000,0.682759,0.514768,0.132075,0.643533,0.424061,0.406504,0.644689,0.600571,0
123,0.531579,1.000000,0.411765,0.561856,0.173913,0.565517,0.487342,0.320755,0.504732,0.112628,0.203252,0.670330,0.072753,1


In [76]:
X = wine_df_s.drop("target", axis=1)
y = pd.DataFrame(wine_df_s["target"])

In [77]:
wine_len = len(wine_df_s)
#全体の20%のデータ行数を求める（Testデータにする用）
wine_split = round(wine_len / 100 * 20)
wine_split

36

In [78]:
X_test = X[:wine_split]
X_train = X[wine_split:]
y_test = y[:wine_split]
y_train = y[wine_split:]

# 交差検証
train, valid を分割しつつ valid をテスト

In [79]:
from sklearn.metrics import accuracy_score # 回答はaccuracy だけにしています．ここも特にしていないです

In [80]:
# k回訓練し，valid で評価
train_len = len(X_train)
sumac = 0
for i in range(k):
    X_valid = X_train[round(train_len*i/k) : round(train_len*(i+1)/k)]
    X_train2 = X_train.drop(X_valid.index)
    y_valid = y_train[round(train_len*i/k) : round(train_len*(i+1)/k)]
    y_train2 = y_train.drop(y_valid.index)
    clf.fit(X_train2, y_train2)
    pred = clf.predict(X_valid)
    ac = accuracy_score(pred, y_valid)
    sumac += ac
    print("accuracy: {}".format(ac))

print("accuracy average: {}".format(sumac/k))

accuracy: 0.9285714285714286
accuracy: 0.9655172413793104
accuracy: 0.9285714285714286
accuracy: 0.896551724137931
accuracy: 0.9642857142857143
accuracy average: 0.9366995073891624


In [81]:
# train すべてで訓練し，test を評価
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("accuracy: {}".format(accuracy_score(pred, y_test)))

accuracy: 0.9722222222222222
